In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, f1_score
from scipy.ndimage.filters import gaussian_filter1d, uniform_filter1d, median_filter
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set_style("darkgrid")

#Import libraries for Deep Learning
from keras.models import Sequential, Model
from keras.layers import Conv1D, LSTM, MaxPool1D, Dense, Dropout, Flatten, BatchNormalization, Input, concatenate, Activation
from keras.optimizers import Adam

print ('Libraries Loaded!')

<ipython-input-1-a698d1c6b9f6>:6: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d, uniform_filter1d, median_filter
<ipython-input-1-a698d1c6b9f6>:6: DeprecationWarning: Please use `uniform_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d, uniform_filter1d, median_filter
<ipython-input-1-a698d1c6b9f6>:6: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d, uniform_filter1d, median_filter


Libraries Loaded!


In [ ]:
def fourier(data):
    datafft= np.zeros(data.shape)
    for row in range(data.shape[0]):
        #fft
        fou=np.fft.fft(data[row])
        f1,f2= 0, 0
        fr=np.fft.fftfreq(3197,0.01)
        #make up a narrow bandpass with a Gaussian
        df=0.1
        gpl= np.exp(- ((fr-f1)/(2*df))**2)+ np.exp(- ((fr-f2)/(2*df))**2)  # pos. frequencies
        gmn= np.exp(- ((fr+f1)/(2*df))**2)+ np.exp(- ((fr+f2)/(2*df))**2)  # neg. frequencies
        g=gpl+gmn
        filt=fou*g  #filtered spectrum = spectrum * bandpass
        #ifft
        s2=np.fft.ifft(filt)
        datafft[row]= s2
    return (datafft)

In [ ]:
data1 = pd.read_csv('ExoTrain.csv')
Y1 = data1['LABEL'].values - 1
X = data1.drop('LABEL', axis=1).values
Y = Y1[:,np.newaxis]

FileNotFoundError: [Errno 2] No such file or directory: 'ExoTrain.csv'

# New section

In [ ]:
label= 1             # Here label i is exoplanet and 0 is non-planet
num_row=10           # Number of rows to plot
col_plot=5           # 5 plots in a row
j=1
train= data1          # plots from training data

print ('Label=', label,'\n\
First Column-      Original\n\
Second Column-     Rolling Mean\n\
Third Column-      Median Filter\n\
Fourth Column-     Uniform Filter\n\
Fifth Column-      Fourier Filter ')
fig = plt.figure(figsize=(8*col_plot, num_row*5))
for i in range(num_row):

    fig.add_subplot(num_row,col_plot,j); j=j+1
    new1= train[train.LABEL==label].iloc[i, :].drop('LABEL')
    new1.plot(color= 'red', label= 'new1')

    fig.add_subplot(num_row,col_plot,j); j=j+1
    new2 = train[train.LABEL==label].iloc[i, :].drop('LABEL').rolling(window=100).mean()
    new2.plot(color= 'blue', label= 'new2')

    fig.add_subplot(num_row,col_plot,j); j=j+1
    new3=median_filter(train[train.LABEL==label].iloc[i, :].drop('LABEL'), size=100)
    plt.plot(new3, color= 'black', label= 'new3')

    fig.add_subplot(num_row,col_plot,j); j=j+1
    new4= uniform_filter1d(train[train.LABEL==label].iloc[i, :].drop('LABEL'), size=100)
    plt.plot(new4, color= 'black', label= 'new4')

    fig.add_subplot(num_row,col_plot,j); j=j+1
    new5= fourier(train[train.LABEL==label].iloc[i, :].drop('LABEL').values[np.newaxis, :])
    plt.plot(new5[0], color= 'black', label= 'new5')

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=531, stratify=Y)

In [ ]:
#Scale each observation to zero mean and unit variance.
x_train = ((x_train - np.mean(x_train, axis=1).reshape(-1,1)) /
           np.std(x_train, axis=1).reshape(-1,1))
x_test = ((x_test - np.mean(x_test, axis=1).reshape(-1,1)) /
          np.std(x_test, axis=1).reshape(-1,1))

In [ ]:
#Noise in the data can be ignored by adding a gaussian filter
#This could more elegantly be done as an extra layer with fixed weights.
x_train1 = np.stack([x_train, gaussian_filter1d(x_train, 1, axis=1)], axis=2)
x_test1 = np.stack([x_test, gaussian_filter1d(x_test, 1, axis=1)], axis=2)

In [ ]:
x_train2 = np.stack([x_train, fourier(x_train)], axis=2)
x_test2 = np.stack([x_test, fourier(x_test)], axis=2)

In [ ]:
x_train3 = x_train[:,:,np.newaxis]
x_test3 = x_test[:,:,np.newaxis]

In [ ]:
print ('Shape of x_train1', x_train1.shape)
print ('Shape of x_test1', x_test1.shape)
print("*"*40)
print ('Shape of x_train2', x_train2.shape)
print ('Shape of x_test2', x_test2.shape)
print("*"*40)
print ('Shape of x_train3', x_train3.shape)
print ('Shape of x_test3', x_test3.shape)

In [ ]:
# 1-D convolutions are frequently used to learn sequential patterns
model1 = Sequential()
model1.add(Conv1D(filters=16, kernel_size=11, activation='relu', input_shape=x_train1.shape[1:]))
model1.add(MaxPool1D(strides=4))
model1.add(BatchNormalization())
model1.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
model1.add(MaxPool1D(strides=4))
model1.add(BatchNormalization())
model1.add(Conv1D(filters=32, kernel_size=11, activation='relu'))
model1.add(MaxPool1D(strides=4))
model1.add(BatchNormalization())
model1.add(Conv1D(filters=32, kernel_size=11, activation='relu'))
model1.add(MaxPool1D(strides=4))
model1.add(Flatten())
model1.add(Dropout(0.3))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
# 1-D convolutions are frequently used to learn sequential patterns
model2 = Sequential()
model2.add(Conv1D(filters=24, kernel_size=11, activation='relu', input_shape=x_train2.shape[1:]))
model2.add(MaxPool1D(strides=4))
model2.add(BatchNormalization())
model2.add(Conv1D(filters=64, kernel_size=7, activation='relu'))
model2.add(MaxPool1D(strides=4))
model2.add(BatchNormalization())
model2.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
model2.add(MaxPool1D(strides=4))
model2.add(BatchNormalization())
model2.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
model2.add(MaxPool1D(strides=4))
model2.add(Flatten())
model2.add(Dropout(0.25))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
model3 = Sequential()
model3.add(Conv1D(filters=16, kernel_size=11, activation='relu', input_shape=x_train3.shape[1:]))
model3.add(MaxPool1D(strides=4))
model3.add(BatchNormalization())
model3.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
model3.add(MaxPool1D(strides=4))
model3.add(BatchNormalization())
model3.add(Conv1D(filters=32, kernel_size=11, activation='relu'))
model3.add(MaxPool1D(strides=4))
model3.add(BatchNormalization())
model3.add(Conv1D(filters=32, kernel_size=11, activation='relu'))
model3.add(MaxPool1D(strides=4))
model3.add(LSTM(10))
model3.add(Dense(1, activation='sigmoid'))

In [ ]:
#x_input= Input(shape= x_train1.shape[1:])
#x= Conv1D(filters=1, kernel_size=11, activation='relu', padding='same')(x_input)
#x=BatchNormalization()(x)

#from keras.layers import concatenate

#m = concatenate([x, x_input])

#x= Conv1D(filters=16, kernel_size=11, activation='relu')(m)
#x=MaxPool1D(strides=4)(x)
#x=BatchNormalization()(x)

#x= Conv1D(filters=16, kernel_size=11, activation='relu')(x)
#x=MaxPool1D(strides=4)(x)
#x=BatchNormalization()(x)

#x= Conv1D(filters=64, kernel_size=11, activation='relu')(x)
#x= MaxPool1D(strides=4)(x)
#x=Flatten()(x)
#x=Dropout(.3)(x)
#x=Dense(32, activation='relu')(x)
#x=Dropout(0.3)(x)
#x=Dense(64, activation='relu')(x)
#x=Dense(1, activation='sigmoid')(x)
#model4= Model(input=x_input, output= x)

#model4.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

from keras.models import Model
from keras.layers import Input, Conv1D, MaxPool1D, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import concatenate

x_input = Input(shape=x_train1.shape[1:])
x = Conv1D(filters=1, kernel_size=11, activation='relu', padding='same')(x_input)
x = BatchNormalization()(x)

m = concatenate([x, x_input])

x = Conv1D(filters=16, kernel_size=11, activation='relu')(m)
x = MaxPool1D(strides=4)(x)
x = BatchNormalization()(x)

x = Conv1D(filters=16, kernel_size=11, activation='relu')(x)
x = MaxPool1D(strides=4)(x)
x = BatchNormalization()(x)

x = Conv1D(filters=64, kernel_size=11, activation='relu')(x)
x = MaxPool1D(strides=4)(x)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model4 = Model(inputs=x_input, outputs=x)

model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model1.summary()

In [ ]:
model2.summary()

In [ ]:
model3.summary()

In [ ]:
model4.summary()

In [ ]:
# feeding training data to the model in batches
def batch_generator(x_train, y_train, batch_size=32):
    """
    Gives equal number of positive and negative samples, and rotates them randomly in time
    """
    half_batch = 16
    x_batch = np.empty((batch_size, x_train.shape[1], x_train.shape[2]), dtype='float32')
    y_batch = np.empty((batch_size, y_train.shape[1]), dtype='float32')

    planet = np.where(y_train[:,0] == 1.)[0]
    non_planet = np.where(y_train[:,0] == 0.)[0]

    while True:
        np.random.shuffle(planet)
        np.random.shuffle(non_planet)

        x_batch[:half_batch] = x_train[planet[:half_batch]]
        x_batch[half_batch:] = x_train[non_planet[half_batch:batch_size]]
        y_batch[:half_batch] = y_train[planet[:half_batch]]
        y_batch[half_batch:] = y_train[non_planet[half_batch:batch_size]]

        for i in range(batch_size):
            roll_state = np.random.randint(x_batch.shape[1])
            x_batch[i] = np.roll(x_batch[i], roll_state, axis = 0)

        yield x_batch, y_batch

In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1_history = model1.fit_generator(batch_generator(x_train1, y_train, 48),
                    validation_data=(x_test1, y_test),
                    verbose=2, epochs=35,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model1.compile(optimizer=Adam(0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])
model1_history2 = model1.fit_generator(batch_generator(x_train1, y_train, 48),
                           validation_data=(x_test1, y_test),
                           verbose=2, epochs=10,
                           steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model1.compile(optimizer=Adam(0.00001), loss = 'binary_crossentropy', metrics=['accuracy'])
model1_history3 = model1.fit_generator(batch_generator(x_train1, y_train, 48),
                    validation_data=(x_test1, y_test),
                    verbose=2, epochs=10,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2_history = model2.fit_generator(batch_generator(x_train2, y_train, 48),
                    validation_data=(x_test2, y_test),
                    verbose=2, epochs=35,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:

model2.compile(optimizer=Adam(0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])
model2_history2 = model2.fit_generator(batch_generator(x_train2, y_train, 48),
                           validation_data=(x_test2, y_test),
                           verbose=2, epochs=5,
                           steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model2.compile(optimizer=Adam(0.00001), loss = 'binary_crossentropy', metrics=['accuracy'])
model2_history3 = model2.fit_generator(batch_generator(x_train2, y_train, 48),
                    validation_data=(x_test2, y_test),
                    verbose=2, epochs=5,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3_history = model3.fit_generator(batch_generator(x_train3, y_train, 48),
                    validation_data=(x_test3, y_test),
                    verbose=2, epochs=35,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
model3_history2 = model3.fit_generator(batch_generator(x_train3, y_train, 48),
                    validation_data=(x_test3, y_test),
                    verbose=2, epochs=10,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer=Adam(0.00001), metrics=['accuracy'])
model3_history3 = model3.fit_generator(batch_generator(x_train3, y_train, 32),
                    validation_data=(x_test3, y_test),
                    verbose=2, epochs=5,
                    steps_per_epoch=x_train.shape[1]//32)


In [ ]:
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4_history = model4.fit_generator(batch_generator(x_train1, y_train, 48),
                    validation_data=(x_test1, y_test),
                    verbose=2, epochs=35,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model4.compile(loss='binary_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
model4_history2 = model4.fit_generator(batch_generator(x_train1, y_train, 48),
                    validation_data=(x_test1, y_test),
                    verbose=2, epochs=10,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
model4.compile(loss='binary_crossentropy', optimizer=Adam(0.00001), metrics=['accuracy'])
model4_history3 = model4.fit_generator(batch_generator(x_train1, y_train, 48),
                    validation_data=(x_test1, y_test),
                    verbose=2, epochs=5,
                    steps_per_epoch=x_train.shape[1]//32)

In [ ]:
# plotting validation loss and accuracy
plt.figure(figsize=(10,6))
plt.plot(model1_history.history['loss'], color='r', label="Model 1 Training loss")
plt.plot(model2_history.history['loss'], color='g', label="Model 2 Training loss")
plt.plot(model3_history.history['loss'], color='b', label="Model 3 Training loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot(model1_history.history['val_loss'], color='r', label="Model 1 Validation loss")
plt.plot(model2_history.history['val_loss'], color='g', label="Model 2 Validation loss")
plt.plot(model3_history.history['val_loss'], color='b', label="Model 3 Validation loss")
plt.legend()
plt.show()

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


plt.figure(figsize=(10,6))
plt.plot(model1_history.history['accuracy'], color='r', label="Model 1 Training accuracy")
plt.plot(model2_history.history['accuracy'], color='g', label="Model 2 Training accuracy")
plt.plot(model3_history.history['accuracy'], color='b', label="Model 3 Training accuracy")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot(model1_history.history['val_accuracy'], color='r', label="Model 1 Validation accuracy")
plt.plot(model2_history.history['val_accuracy'], color='g', label="Model 2 Validation accuracy")
plt.plot(model3_history.history['val_accuracy'], color='b', label="Model 3 Validation accuracy")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(model4_history.history['loss'], color='m', label="Training loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot(model4_history.history['val_loss'], color='m', label="Validation loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot(model4_history.history['accuracy'], color='m', label="Training accuracy")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot(model4_history.history['val_accuracy'], color='m', label="Validation accuracy")
plt.legend()
plt.show()

In [ ]:
# obtaining the indexes of yes-exoplanet and no-exoplanet
no_exoplanet = np.where(y_test[:,0] == 0.)[0]
yes_exoplanet = np.where(y_test[:,0] == 1.)[0]

In [ ]:
y_pred1 = model1.predict(x_test1)[:,0]

y_pred2 = model2.predict(x_test2)[:,0]

y_pred3 = model3.predict(x_test3)[:,0]

y_pred4 = model4.predict(x_test1)[:,0]

In [ ]:
# plots indicating the model allotting different range of scores for different classes
# thus imploying the precision of the model
plt.figure(figsize=(10,6))
plt.plot([y_pred1[i] for i in yes_exoplanet], 'ro', label="Exoplanet")
plt.plot([y_pred2[i] for i in yes_exoplanet], 'go', label="Exoplanet")
plt.plot([y_pred3[i] for i in yes_exoplanet], 'bo', label="Exoplanet")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot([y_pred1[i] for i in no_exoplanet], 'ro', label="Non-exoplanet")
plt.plot([y_pred2[i] for i in no_exoplanet], 'go', label="Non-exoplanet")
plt.plot([y_pred3[i] for i in no_exoplanet], 'bo', label="Non-exoplanet")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot([y_pred4[i] for i in yes_exoplanet], 'mo', label="Exoplanet")
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.plot([y_pred4[i] for i in no_exoplanet], 'mo', label="Non-exoplanet")
plt.legend()
plt.show()

In [ ]:
y_true = (y_test[:, 0] + 0.5).astype("int")
f1_score_= []
skill_score_= []

for i in (np.arange(0.5,1.0,.005)):
    y_pred_int= np.copy(y_pred1)
    y_pred_int[(y_pred_int>i)]=1
    y_pred_int[(y_pred_int<=i)]=0

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_int).ravel()

    skill_score_.append(((tp * tn) - (fp * fn)) / ((tp + fn) * (fp + tn)))
    f1_score_.append(f1_score(y_true, y_pred_int, average='macro'))



print("F1 score : ", np.max(f1_score_))
print("Threshold for F1 score : ", (np.min(np.where(f1_score_ == np.max(f1_score_)))+1)*0.005+0.5)
print("*"*40)
print("True skill score : ", np.max(skill_score_))
print("Threshold for True skill score : ", (np.min(np.where(skill_score_ == np.max(skill_score_)))+1)*0.005+0.5)

plt.figure(figsize=(10,6))
plt.plot(f1_score_, color='b', label="F1 score")
plt.plot(skill_score_, color='r', label="True skill score")
plt.legend()
plt.show()

In [ ]:
y_true = (y_test[:, 0] + 0.5).astype("int")
f1_score_= []
skill_score_= []

for i in (np.arange(0.5,1.0,.005)):
    y_pred_int= np.copy(y_pred2)
    y_pred_int[(y_pred_int>i)]=1
    y_pred_int[(y_pred_int<=i)]=0

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_int).ravel()

    skill_score_.append(((tp * tn) - (fp * fn)) / ((tp + fn) * (fp + tn)))
    f1_score_.append(f1_score(y_true, y_pred_int, average='macro'))



print("F1 score : ", np.max(f1_score_))
print("Threshold for F1 score : ", (np.min(np.where(f1_score_ == np.max(f1_score_)))+1)*0.005+0.5)
print("*"*40)
print("True skill score : ", np.max(skill_score_))
print("Threshold for True skill score : ", (np.min(np.where(skill_score_ == np.max(skill_score_)))+1)*0.005+0.5)

plt.figure(figsize=(10,6))
plt.plot(f1_score_, color='b', label="F1 score")
plt.plot(skill_score_, color='r', label="True skill score")
plt.legend()
plt.show()

In [ ]:
y_true = (y_test[:, 0] + 0.5).astype("int")
f1_score_= []
skill_score_= []

for i in (np.arange(0.5,1.0,.005)):
    y_pred_int= np.copy(y_pred3)
    y_pred_int[(y_pred_int>i)]=1
    y_pred_int[(y_pred_int<=i)]=0

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_int).ravel()

    skill_score_.append(((tp * tn) - (fp * fn)) / ((tp + fn) * (fp + tn)))
    f1_score_.append(f1_score(y_true, y_pred_int, average='macro'))



print("F1 score : ", np.max(f1_score_))
print("Threshold for F1 score : ", (np.min(np.where(f1_score_ == np.max(f1_score_)))+1)*0.005+0.5)
print("*"*40)
print("True skill score : ", np.max(skill_score_))
print("Threshold for True skill score : ", (np.min(np.where(skill_score_ == np.max(skill_score_)))+1)*0.005+0.5)

plt.figure(figsize=(10,6))
plt.plot(f1_score_, color='b', label="F1 score")
plt.plot(skill_score_, color='r', label="True skill score")
plt.legend()
plt.show()

In [ ]:
y_true = (y_test[:, 0] + 0.5).astype("int")
f1_score_= []
skill_score_= []

for i in (np.arange(0.5,1.0,.005)):
    y_pred_int= np.copy(y_pred4)
    y_pred_int[(y_pred_int>i)]=1
    y_pred_int[(y_pred_int<=i)]=0

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_int).ravel()

    skill_score_.append(((tp * tn) - (fp * fn)) / ((tp + fn) * (fp + tn)))
    f1_score_.append(f1_score(y_true, y_pred_int, average='macro'))



print("F1 score : ", np.max(f1_score_))
print("Threshold for F1 score : ", (np.min(np.where(f1_score_ == np.max(f1_score_)))+1)*0.005+0.5)
print("*"*40)
print("True skill score : ", np.max(skill_score_))
print("Threshold for True skill score : ", (np.min(np.where(skill_score_ == np.max(skill_score_)))+1)*0.005+0.5)

plt.figure(figsize=(10,6))
plt.plot(f1_score_, color='b', label="F1 score")
plt.plot(skill_score_, color='r', label="True skill score")
plt.legend()
plt.show()

In [ ]:
for i in range(len(y_pred1)):
    if y_pred1[i]>0.97: # this value is obtained from the above plot
        y_pred1[i] = 1
    else:
        y_pred1[i] = 0

for i in range(len(y_pred2)):
    if y_pred2[i]>0.505: # this value is obtained from the above plot
        y_pred2[i] = 1
    else:
        y_pred2[i] = 0

for i in range(len(y_pred3)):
    if y_pred3[i]>=1: # this value is obtained from the above plot
        y_pred3[i] = 1
    else:
        y_pred3[i] = 0

for i in range(len(y_pred4)):
    if y_pred4[i]>=0.97: # this value is obtained from the above plot
        y_pred4[i] = 1
    else:
        y_pred4[i] = 0

In [ ]:
# the precision recall of each class is separately shown hence
# removing the ambiguity of imbalanced classes..
from sklearn.metrics import classification_report
print("Model 1st report")
print(classification_report(y_true, y_pred1, target_names=['Explanet','Not Explanet']))
print("*"*80)
print("Model 2st report")
print(classification_report(y_true, y_pred2, target_names=['Explanet','Not Explanet']))
print("*"*80)
print("Model 3st report")
print(classification_report(y_true, y_pred3, target_names=['Explanet','Not Explanet']))
print("*"*80)
print("Model 4st report")
print(classification_report(y_true, y_pred4, target_names=['Explanet','Not Explanet']))

https://github.com/khumairraj/Exoplanet-Detection/blob/master/Code/.ipynb_checkpoints/local_test_exoplanet_detection-checkpoint.ipynb